In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

## 總表讀取

In [60]:
import requests
import pandas as pd

url = "http://192.168.31.130:32327/games_reviews/_search?scroll=1m"  # 使用 Scroll，scroll=1m 表示 Scroll 保持活躍1分鐘
headers = {"Content-Type": "application/json"}

# 初始化 Scroll 查詢，指定要返回的欄位
data = {
    "_source": [
        "steamId",
        "tags",
        "genres",
        "features",
        "languages",
        "developers",
        "publishers",
    ],  # 查詢指定的欄位
    "query": {"match_all": {}},  # 匹配所有文件
    "size": 1000,  # 每次返回1000筆資料
}

response = requests.get(url, headers=headers, json=data)
response_json = response.json()

# 提取初始查詢的 hits 和 scroll_id
hits = response_json["hits"]["hits"]
scroll_id = response_json["_scroll_id"]

# 存儲資料的列表
data_list = [
    {
        "steamId": hit["_source"].get("steamId"),
        "tags": hit["_source"].get("tags"),
        "genres": hit["_source"].get("genres"),
        "features": hit["_source"].get("features"),
        "languages": hit["_source"].get("languages"),
        "developers": hit["_source"].get("developers"),
        "publishers": hit["_source"].get("publishers"),
    }
    for hit in hits
]

# 使用 Scroll API 繼續獲取剩餘的資料
while len(hits) > 0:
    scroll_url = "http://192.168.31.130:32327/_search/scroll"
    scroll_data = {
        "scroll": "1m",  # Scroll 保持活躍1分鐘
        "scroll_id": scroll_id,  # 使用前次查詢返回的 scroll_id 繼續查詢
    }

    scroll_response = requests.get(scroll_url, headers=headers, json=scroll_data)
    scroll_response_json = scroll_response.json()

    hits = scroll_response_json["hits"]["hits"]
    scroll_id = scroll_response_json["_scroll_id"]

    # 將新獲取的 hits 資料追加到列表中
    data_list.extend(
        [
            {
                "steamId": hit["_source"].get("steamId"),
                "tags": hit["_source"].get("tags"),
                "genres": hit["_source"].get("genres"),
                "features": hit["_source"].get("features"),
                "languages": hit["_source"].get("languages"),
                "developers": hit["_source"].get("developers"),
                "publishers": hit["_source"].get("publishers"),
            }
            for hit in hits
        ]
    )

# 將提取的資料轉換為 DataFrame
df = pd.DataFrame(data_list)

# 顯示 DataFrame
df

,steamId,tags,genres,features,languages,developers,publishers
0,240,"[Shooter, FPS, Action, Multiplayer, Team-Based...",[Action],"[Cross-Platform Multiplayer, Steam Achievement...","[English, French, German, Italian, Japanese, K...",[Valve],[Valve]
1,80,"[Action, FPS, Shooter, Multiplayer, First-Pers...",[Action],"[Single-player, Valve Anti-Cheat enabled, Fami...","[English, French, German, Italian, Spanish - S...",[Valve],[Valve]
2,262280,"[Strategy, RPG, Management, Fantasy, Dungeon C...","[RPG, Simulation, Strategy]","[Single-player, Cross-Platform Multiplayer, St...","[English, German, French, Italian, Spanish - S...",[Realmforge Studios],[Kalypso Media Digital]
3,1179680,"[Free to Play, Sandbox, Massively Multiplayer,...","[Adventure, Free To Play, Indie, Massively Mul...","[MMO, Online PvP, Online Co-op, Cross-Platform...",[English],[Code Club AB],[Code Club AB]
4,688130,"[Psychological Horror, Precision Platformer, D...","[Action, Adventure, Casual, Indie]","[Single-player, Online PvP, Online Co-op, Stea...","[English, French, Italian, German, Spanish - S...",[Hendrik Felix Pohl],[Hendrik Felix Pohl]
...,...,...,...,...,...,...,...
10331,2195410,"[Early Access, Atmospheric, Singleplayer, Mult...","[Indie, Strategy, Early Access]","[Single-player, Online PvP, Online Co-op, Stea...","[English, French, German, Spanish - Spain, Jap...",[Chimera Entertainment],"[Chimera Entertainment, H2 Interactive (Korea ..."
10332,2293680,"[Survival, Crafting, Adventure, Open World Sur...","[Action, Adventure, Indie, Massively Multiplay...","[Single-player, MMO, Online PvP, Online Co-op,...",[English],[Bloodbath Studios],[Bloodbath Studios]
10333,262100,"[Action, Casual, Indie, Pixel Graphics, Destru...","[Action, Casual, Indie]","[Single-player, Online PvP, Shared/Split Scree...",[English],[Transhuman Design],[Transhuman Design]
10334,1277870,"[Competitive, Action, Casual, Physics, Colorfu...","[Action, Casual, Sports]","[Single-player, Online PvP, Shared/Split Scree...","[English, Japanese, Simplified Chinese]",[VV-LABO],[Phoenixx Inc.]


In [30]:
feature = df.columns[1:]
feature

Index(['tags', 'genres', 'features', 'languages', 'developers', 'publishers'], dtype='object')

### 總表前處理，將儲存格內的list資料做清洗。

In [31]:
df_combined = pd.DataFrame({"steamId": df["steamId"]})

for column in feature:


    # 將 tags 欄位的列表轉換為字符串，並去除 , 和 [] 等符號

    df_re = df[column].apply(
        lambda x: ",".join(x).replace("[", "").replace("]", "").strip().strip('"')
    )


    df_combined[column] = df_re

df_combined

,steamId,tags,genres,features,languages,developers,publishers
0,240,"Shooter,FPS,Action,Multiplayer,Team-Based,Firs...",Action,"Cross-Platform Multiplayer,Steam Achievements,...","English,French,German,Italian,Japanese,Korean,...",Valve,Valve
1,80,"Action,FPS,Shooter,Multiplayer,First-Person,Si...",Action,"Single-player,Valve Anti-Cheat enabled,Family ...","English,French,German,Italian,Spanish - Spain,...",Valve,Valve
2,262280,"Strategy,RPG,Management,Fantasy,Dungeon Crawle...","RPG,Simulation,Strategy","Single-player,Cross-Platform Multiplayer,Steam...","English,German,French,Italian,Spanish - Spain,...",Realmforge Studios,Kalypso Media Digital
3,1179680,"Free to Play,Sandbox,Massively Multiplayer,RPG...","Adventure,Free To Play,Indie,Massively Multipl...","MMO,Online PvP,Online Co-op,Cross-Platform Mul...",English,Code Club AB,Code Club AB
4,688130,"Psychological Horror,Precision Platformer,Diff...","Action,Adventure,Casual,Indie","Single-player,Online PvP,Online Co-op,Steam Ac...","English,French,Italian,German,Spanish - Spain,...",Hendrik Felix Pohl,Hendrik Felix Pohl
...,...,...,...,...,...,...,...
10331,2195410,"Early Access,Atmospheric,Singleplayer,Multipla...","Indie,Strategy,Early Access","Single-player,Online PvP,Online Co-op,Steam Ac...","English,French,German,Spanish - Spain,Japanese...",Chimera Entertainment,"Chimera Entertainment,H2 Interactive (Korea an..."
10332,2293680,"Survival,Crafting,Adventure,Open World Surviva...","Action,Adventure,Indie,Massively Multiplayer,R...","Single-player,MMO,Online PvP,Online Co-op,Fami...",English,Bloodbath Studios,Bloodbath Studios
10333,262100,"Action,Casual,Indie,Pixel Graphics,Destruction...","Action,Casual,Indie","Single-player,Online PvP,Shared/Split Screen P...",English,Transhuman Design,Transhuman Design
10334,1277870,"Competitive,Action,Casual,Physics,Colorful,Cut...","Action,Casual,Sports","Single-player,Online PvP,Shared/Split Screen P...","English,Japanese,Simplified Chinese",VV-LABO,Phoenixx Inc.


### (補)以下為自動化生成所有表格內容(包含tags、genres、languages、features、publishers、developers)，每張表格接包含steamId。

In [32]:
# 獲取指定的欄位範圍
df_columns = df.columns[1:]

# 確保目標目錄存在，若不存在則創建
output_directory = "./data/"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

for column in df_columns:
    df_tags = df[["steamId", column]]

    # 將 tags 欄位的列表轉換為字符串，並去除 , 和 [] 等符號
    df_split = df_tags[column].apply(
        lambda x: ",".join(x).replace("[", "").replace("]", "").strip().strip('"')
    )

    # 使用 get_dummies 進行 one-hot 編碼
    df_dummies = df_split.str.get_dummies(sep=",")

    # 將 one-hot 編碼後的標籤與原始的 DataFrame 合併
    df_final = pd.concat([df_tags.drop(column, axis=1), df_dummies], axis=1)

    # 將結果保存為 JSON 文件，並根據 column 命名文件
    json_filename = f"id_{column}.json"

    # 將最後的內容存成 json，並將檔案儲存在 data 的資料夾中
    df_final.to_json(os.path.join(output_directory, json_filename), orient="records")

## 以下為各自生成小表格

## tags 表格

In [62]:
df_tags = df[["steamId", "tags"]]

# 首先將 `tags` 欄位的列表轉換為字符串格式，然後用 `str.get_dummies()` 做 one-hot 編碼
df_tags["tags"] = df_tags["tags"].apply(lambda x: ",".join(x))

# 使用 get_dummies 進行 one-hot 編碼
tags_dummies = df_tags["tags"].str.get_dummies(sep=",")

# 將 one-hot 編碼後的標籤與原始的 DataFrame 合併
df_tags = pd.concat([df_tags.drop("tags", axis=1), tags_dummies], axis=1)

C:\Users\student\AppData\Local\Temp\ipykernel_16988\598758924.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tags["tags"] = df_tags["tags"].apply(lambda x: ",".join(x))


In [63]:
df_tags.head()

,steamId,1980s,1990's,2.5D,2D,2D Fighter,2D Platformer,360 Video,3D,3D Fighter,...,Well-Written,Werewolves,Western,Wholesome,Word Game,World War I,World War II,Wrestling,Zombies,eSports
0,240,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,80,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,262280,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1179680,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,688130,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [74]:
df_tags

,steamId,1980s,1990's,2.5D,2D,2D Fighter,2D Platformer,360 Video,3D,3D Fighter,...,Well-Written,Werewolves,Western,Wholesome,Word Game,World War I,World War II,Wrestling,Zombies,eSports
0,240,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,80,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,262280,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1179680,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,688130,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10331,2195410,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
10332,2293680,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10333,262100,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
10334,1277870,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## genres 表格


In [38]:
df_genres = df[["steamId", "genres"]]
df_genres["genres"] = df_genres["genres"].apply(lambda x: ",".join(x))
genres_dummies = df_genres["genres"].str.get_dummies(sep=",")
df_genres = pd.concat([df_genres.drop("genres", axis=1), genres_dummies], axis=1)

C:\Users\student\AppData\Local\Temp\ipykernel_16988\2998761292.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genres["genres"] = df_genres["genres"].apply(lambda x: ",".join(x))


In [39]:
df_genres.head()

,steamId,Accounting,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Early Access,Education,...,Massively Multiplayer,Movie,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Web Publishing
0,240,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,80,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,262280,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0
3,1179680,0,0,1,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
4,688130,0,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## features 表格


In [40]:
df_features = df[["steamId", "features"]]
df_features["features"] = df_features["features"].apply(lambda x: ",".join(x))
features_dummies = df_features["features"].str.get_dummies(sep=",")
df_features = pd.concat(
    [df_features.drop("features", axis=1), features_dummies], axis=1
)

C:\Users\student\AppData\Local\Temp\ipykernel_16988\642846041.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features["features"] = df_features["features"].apply(lambda x: ",".join(x))


In [41]:
df_features.head()

,steamId,Captions available,Commentary available,Cross-Platform Multiplayer,Family Sharing,HDR available,In-App Purchases,Includes Source SDK,Includes level editor,LAN Co-op,...,Steam Cloud,Steam Leaderboards,Steam Timeline,Steam Trading Cards,Steam Turn Notifications,SteamVR Collectibles,Tracked Controller Support,VR Only,VR Supported,Valve Anti-Cheat enabled
0,240,0,0,1,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
1,80,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,262280,0,0,1,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
3,1179680,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,688130,0,0,0,1,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


## languages 表格


In [42]:
df_languages = df[["steamId", "languages"]]
df_languages["languages"] = df_languages["languages"].apply(lambda x: ",".join(x))
languages_dummies = df_languages["languages"].str.get_dummies(sep=",")
df_languages = pd.concat(
    [df_languages.drop("languages", axis=1), languages_dummies], axis=1
)

C:\Users\student\AppData\Local\Temp\ipykernel_16988\3496918455.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_languages["languages"] = df_languages["languages"].apply(lambda x: ",".join(x))


In [43]:
df_languages.head()

,steamId,#lang_slovakian,Afrikaans,Albanian,Amharic,Arabic,Armenian,Assamese,Azerbaijani,Bangla,...,Urdu,Uyghur,Uzbek,Valencian,Vietnamese,Welsh,Wolof,Xhosa,Yoruba,Zulu
0,240,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,80,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,262280,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1179680,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,688130,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## developers 表格


In [44]:
df_developers = df[["steamId", "developers"]]
df_developers["developers"] = df_developers["developers"].apply(lambda x: ",".join(x))
developers_dummies = df_developers["developers"].str.get_dummies(sep=",")
df_developers = pd.concat(
    [df_developers.drop("developers", axis=1), developers_dummies], axis=1
)

C:\Users\student\AppData\Local\Temp\ipykernel_16988\2340714100.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_developers["developers"] = df_developers["developers"].apply(lambda x: ",".join(x))


In [45]:
df_developers.head()

,steamId,Scott Greene,Arttu Pekkala,Bucharest,Capu109,Co.,Deborah Spruill,INC,Inc,Inc.,...,香港美心,香蜂草工作室,高野千晴 Takano Chiharu,魔书科技,魔龙之戒,黄昏フロンティア,黑暗火鍋,龙游工作室,（Hong Kong）GKD,（Hong Kong）GKD Game Studio
0,240,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,80,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,262280,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1179680,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,688130,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## publishers 表格


In [46]:
df_publishers = df[["steamId", "publishers"]]
df_publishers["publishers"] = df_publishers["publishers"].apply(lambda x: ",".join(x))
publishers_dummies = df_publishers["publishers"].str.get_dummies(sep=",")
df_publishers = pd.concat(
    [df_publishers.drop("publishers", axis=1), publishers_dummies], axis=1
)

C:\Users\student\AppData\Local\Temp\ipykernel_16988\1907273851.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_publishers["publishers"] = df_publishers["publishers"].apply(lambda x: ",".join(x))


In [47]:
df_publishers.head()

,steamId,ACE,Co.,INC,INC.,Inc,Inc.,L.L.C.,LIMITED,LLC,...,飞仙诀(福利版)运营团队,香港美心,香蜂草工作室,魔书科技,魔龙之戒,黑暗火鍋,龙游工作室,（Hong Kong）GKD,（Hong Kong）GKD Game Studio,🚀 Raiser Games
0,240,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,80,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,262280,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1179680,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,688130,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
